In [ ]:
import os
import re
import sys
import glob
import pickle
from collections import OrderedDict

import numpy as np
from scipy.optimize import curve_fit
import matplotlib
from matplotlib import rcParams
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow import keras

if '..' not in sys.path:
    sys.path.append('..')
from dlml.utils import collect_experiments
from dlml.data import read_area_values, load_data_slide
from dlml.nn import predict

#### Find the best experiment given a set of tags

In [ ]:
area_ID = 1
area_measure = 'momentum'
stoch_load_bus_IDs = []
rec_bus_IDs = [3, 14, 17, 39]
H_G1, D, DZA = None, None, None # 500, 2, 0
additional_tags = ['ReLU_none', 'converted_from_PowerFactory', 'all_stoch_loads']

experiments = collect_experiments(area_ID, area_measure=area_measure, D=D, DZA=DZA, \
                                  stoch_load_bus_IDs=stoch_load_bus_IDs, H_G1=H_G1, \
                                  rec_bus_IDs=rec_bus_IDs, additional_tags=additional_tags, \
                                  verbose=True)
experiment_IDs = list(experiments.keys())
experiment_ID = experiment_IDs[np.argmin([expt['val_loss'].min() for expt in experiments.values()])]
experiment_ID = '7d55f784f6b64f2caeb866804bda1a8b'
MAPE = experiments[experiment_ID]['MAPE']
loss = experiments[experiment_ID]['loss']
val_loss = experiments[experiment_ID]['val_loss']
batch_loss = experiments[experiment_ID]['batch_loss']
tags = experiments[experiment_ID]['tags']
print(f'The best experiment is {experiment_ID[:6]} (val_loss = {val_loss.min():.4f}, MAPE = {MAPE:.4f}%).')

In [ ]:
experiments_path = '../experiments/neural_network/'
checkpoint_path = experiments_path + experiment_ID + '/checkpoints/'
checkpoint_files = glob.glob(checkpoint_path + '*.h5')
if len(checkpoint_files) == 1:
    best_checkpoint = checkpoint_files[0]
else:
    epochs = [int(os.path.split(file)[-1].split('.')[1].split('-')[0]) for file in checkpoint_files]
    best_checkpoint = checkpoint_files[epochs.index(np.argmin(val_loss) + 1)]
model = keras.models.load_model(best_checkpoint, compile=False)
model.compile()
network_parameters = pickle.load(open(os.path.join(experiments_path, experiment_ID, 'parameters.pkl'), 'rb'))
data_dirs = [os.path.join('..', d.format(a)) if '{}' in d else os.path.join('..', d) \
             for d,a in zip(network_parameters['data_dirs'], network_parameters['area_IDs'])]
# we need mean and standard deviation of the training set to normalize the data
x_train_mean = network_parameters['x_train_mean']
x_train_std  = network_parameters['x_train_std']
data_dir = data_dirs[0]
tmp = [re.findall('.*_bus', var_name)[0] for var_name in network_parameters['var_names'] if 'bus' in var_name]
var_names_fmt = OrderedDict({k + '{}': [] for k in tmp})
tmp = [re.findall('.*_line', var_name)[0] for var_name in network_parameters['var_names'] if 'line' in var_name]
for k in tmp:
    var_names_fmt[k + '_{}_{}'] = []
var_names_fmt = list(var_names_fmt.keys())
if len(rec_bus_IDs) == 0:
    rec_bus_IDs = list(np.unique([int(re.findall('\d+', var_name)[0]) \
                                  for var_name in network_parameters['var_names']]))
    rec_bus_list = 'buses_' + '-'.join(map(str, rec_bus_IDs))
if not os.path.isdir(data_dir):
    raise Exception(f'{data_dir}: no such directory')
print(f'Loaded network from {best_checkpoint}.')
print(f'Data directory is {data_dir}.')
print(f'Variable names: {var_names_fmt}')

In [ ]:
keras.utils.plot_model(model, show_shapes=False, dpi=96)

In [ ]:
default_H = OrderedDict([
    ('G01', 5.00), ('G02', 4.33), ('G03', 4.47), ('G04', 3.57), ('G05', 4.33),
    ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.20)
])
default_H_with_comp = OrderedDict([
    ('G01', 5.00), ('G02', 4.33), ('G03', 4.47), ('G04', 3.57), ('G05', 4.33),
    ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.20),
    ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
])


generators_areas_map = {
    'default': [
        ['G02', 'G03'],
        ['G04', 'G05', 'G06', 'G07'],
        ['G08', 'G09', 'G10'],
        ['G01']
    ]
}
generators_areas_map_with_comp = {
    'default': [
        ['G02', 'G03', 'Comp11'],
        ['G04', 'G05', 'G06', 'G07', 'Comp21'],
        ['G08', 'G09', 'G10', 'Comp31'],
        ['G01']
    ]
}

P_nom = {'G01': 10000e6, 'G02': 700e6, 'G03': 800e6, 'G04':  800e6, 'G05':  300e6,
         'G06':   800e6, 'G07': 700e6, 'G08': 700e6, 'G09': 1000e6, 'G10': 1000e6}
P_nom_with_comp = {'G01': 10000e6, 'G02': 700e6, 'G03': 800e6, 'G04':  800e6, 'G05':  300e6,
                   'G06':   800e6, 'G07': 700e6, 'G08': 700e6, 'G09': 1000e6, 'G10': 1000e6,
                   'Comp11': 100e6, 'Comp21': 100e6, 'Comp31': 100e6}


window_dur = 60
window_step = 1

var_names = network_parameters['var_names']
data_mean = {var_name: x_train_mean[k] for k,var_name in enumerate(var_names)}
data_std = {var_name: x_train_std[k] for k,var_name in enumerate(var_names)}

if area_measure == 'inertia':
    measure_units = 's'
elif area_measure == 'energy':
    measure_units = r'GW$\cdot$s'
elif area_measure == 'momentum':
    measure_units = r'GW$\cdot$s$^2$'
    
stoch_load_bus_list = 'stoch_load_bus_' + '-'.join(map(str, stoch_load_bus_IDs))
rec_bus_list = 'buses_' + '-'.join(map(str, rec_bus_IDs))

abbrv = {'inertia': 'H', 'energy': 'E', 'momentum': 'M'}

## Constant measure

In [ ]:
t[-1]

In [ ]:
H_values = [
    default_H_with_comp,
    OrderedDict([
        ('G01', 5.00), ('G02', 3.83), ('G03', 3.97), ('G04', 3.57), ('G05', 4.33),
        ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
        ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
    ]),
    OrderedDict([
        ('G01', 5.00), ('G02', 4.83), ('G03', 4.97), ('G04', 3.57), ('G05', 4.33),
        ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
        ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
    ])
]
N_H = len(H_values)


measure_exact = []

data_normalized = []
measure = []
area_inertia = []

for H in H_values:
    data_file = data_dir + '/ieee39_PF_stoch_loads_compensators_' + \
        '_'.join(map(lambda h: f'{h:.3f}', H.values())) + '.h5'
    print(f'Reading data from {data_file}...')
    _,_,v,_ = read_area_values(data_file, generators_areas_map['default'], P_nom, area_measure)
    _,_,h,_ = read_area_values(data_file, generators_areas_map['default'], P_nom, 'inertia')
    measure_exact.append(v[area_ID - 1])
    area_inertia.append(h[area_ID - 1])

    t, _, data_norm, data_sliding, _ = load_data_slide([data_file],
                                                        var_names,
                                                        None,
                                                        data_std,
                                                        window_dur,
                                                        window_step,
                                                        add_omega_ref = False,
                                                        verbose = True)
    print(f'Duration of the simulation: {t[-1]} sec.')
    data_normalized.append(data_norm)
    dt = np.diff(t[:2])[0]
    time, HH, _ = predict(model, data_sliding, window_step)
    measure.append(HH)
measure_exact = np.array(measure_exact)
area_inertia = np.array(area_inertia)
measure_predicted = np.array(list(map(np.nanmean, measure)))

In [ ]:
area_inertia

In [ ]:
measure_exact

In [ ]:
measure_predicted

In [ ]:
n_vars = len(var_names_fmt)
fig = plt.figure(figsize=(8, n_vars * 4))
if N_H < 3:
    gs = fig.add_gridspec(n_vars+1, 4)
elif N_H == 3:
    gs = fig.add_gridspec(n_vars+1, 6)
else:
    raise Exception('Supported values of N_H are 1, 2 or 3')
ax = []
for i in range(n_vars):
    if N_H < 3:
        ax.append([fig.add_subplot(gs[i, :3]), fig.add_subplot(gs[i, 3])])
    else:
        ax.append([fig.add_subplot(gs[i, :4]), fig.add_subplot(gs[i, 4:])])
if N_H < 3:
    ax.append([fig.add_subplot(gs[-1, :2]), fig.add_subplot(gs[-1, 2:])])
else:
    ax.append([fig.add_subplot(gs[-1, :2]), fig.add_subplot(gs[-1, 2:4]), fig.add_subplot(gs[-1, 4:])])

col = [[.2,.2,.2], [.8,0,0], [0,.7,0]]

bus_ID = rec_bus_IDs[0]
line_IDs = rec_bus_IDs[0], rec_bus_IDs[0] + 1

idx = t < 60

dm = np.max(measure_exact) - np.min(measure_exact)
ylim = [np.min(measure_exact) - dm / 2, np.max(measure_exact) + dm / 2]

for i in range(N_H):
    for j,var_name in enumerate(var_names_fmt):
        if 'bus' in var_name:
            key = var_name.format(bus_ID)
        elif 'line' in var_name:
            key = var_name.format(line_IDs[0], line_IDs[1])
        value = data_normalized[i][key]
        n,edges = np.histogram(value, bins=25, range=(-4,4), density=True)
        ax[j][0].plot(t[idx], value[idx], color=col[i], lw=1, \
                      label=f'{abbrv[area_measure]} = {measure_exact[i]:.2f} {measure_units}')
        ax[j][1].plot(n, edges[:-1] + np.diff(edges[:2])[0] / 2, color=col[i], lw=1)
        for a in ax[j]:
            a.set_ylim([-4,4])
        ax[j][0].set_ylabel(key)
    ax[-1][i].plot(time / 60, measure[i], 'k', lw=1)
    ax[-1][i].plot(time[[0,-1]] / 60, measure_exact[i] + np.zeros(2), 'k--', lw=1)
    ax[-1][i].set_ylim(ylim)
    ax[-1][i].set_xlabel('Time [min]')

for a in ax:
    for side in 'right','top':
        for aa in a:
            aa.spines[side].set_visible(False)

ax[0][0].legend(loc='best')
# ax[-1][0].set_ylim([0.2, 0.25])
for i in range(1, N_H):
    ax[-1][i].set_xlim(ax[-1][0].get_xlim())
    ax[-1][i].set_ylim(ax[-1][0].get_ylim())
#     ax[-1][0].get_shared_x_axes().join(ax[-1][0], ax[-1][i])

# ax[-1][0].set_xlim([0,30])
ax[-1][0].set_ylabel(f'{area_measure.capitalize()} [{measure_units}]')
ax[-2][0].set_xlabel('Time [s]')
ax[-2][1].set_xlabel('Fraction')
fig.tight_layout()
# output_filename = f'IEEE39_area{area_ID}_H_G1={H_G1}_' + \
#     f'rec_buses={rec_bus_list}_load_buses={stoch_load_bus_list}_const_H_{experiment_ID[:6]}.pdf'
# fig.savefig(output_filename)

### Different combinations of inertia corresponding to the same area momentum

In [ ]:
momentum = lambda H, S, fn=60.: 2 * H@S / fn * 1e-3
S = np.array([700, 800, 100])
H2 = np.linspace(3.33, 5.33, 6)
H3 = np.linspace(3.47, 5.47, 6)
dH = 0.2
H = np.array([4.33, 4.47, 0.1])
print('H = {} -> M = {:.4f} GW.s2'.format(H, momentum(H, S)))
H = np.array([H2[2], H3[3], 0.1]) + np.array([-dH, dH, 0])
print('H = {} -> M = {:.4f} GW.s2'.format(H, momentum(H, S)))
H = np.array([H2[3], H3[2], 0.1]) + np.array([dH, -dH, 0])
print('H = {} -> M = {:.4f} GW.s2'.format(H, momentum(H, S)))

#### multiple files

In [ ]:
H_values = [
    default_H_with_comp,
    OrderedDict([
        ('G01', 5.00), ('G02', 3.93), ('G03', 4.87), ('G04', 3.57), ('G05', 4.33),
        ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
        ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
    ]),
    OrderedDict([
        ('G01', 5.00), ('G02', 4.73), ('G03', 4.07), ('G04', 3.57), ('G05', 4.33),
        ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
        ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
    ])
]
N_H = len(H_values)


measure_exact = []

data_normalized = []
measure = []
area_inertia = []

for H in H_values:
    data_file = data_dir + '/ieee39_PF_stoch_loads_compensators_' + \
        '_'.join(map(lambda h: f'{h:.3f}', H.values())) + '.h5'
    print(f'Reading data from {data_file}...')
    _,_,v,_ = read_area_values(data_file, generators_areas_map['default'], P_nom, area_measure)
    _,_,h,_ = read_area_values(data_file, generators_areas_map['default'], P_nom, 'inertia')
    measure_exact.append(v[area_ID - 1])
    area_inertia.append(h[area_ID - 1])

    t, _, data_norm, data_sliding, _ = load_data_slide([data_file],
                                                        var_names,
                                                        None,
                                                        data_std,
                                                        window_dur,
                                                        window_step,
                                                        add_omega_ref = False,
                                                        verbose = True)
    print(f'Duration of the simulation: {t[-1]} sec.')
    data_normalized.append(data_norm)
    dt = np.diff(t[:2])[0]
    time, HH, _ = predict(model, data_sliding, window_step)
    measure.append(HH)
measure_exact = np.array(measure_exact)
area_inertia = np.array(area_inertia)
measure_predicted = np.array(list(map(np.nanmean, measure)))

In [ ]:
area_inertia

In [ ]:
measure_exact

In [ ]:
measure_predicted

In [ ]:
n_vars = len(var_names_fmt)
fig = plt.figure(figsize=(8, n_vars * 4))
if N_H < 3:
    gs = fig.add_gridspec(n_vars+1, 4)
elif N_H == 3:
    gs = fig.add_gridspec(n_vars+1, 6)
else:
    raise Exception('Supported values of N_H are 1, 2 or 3')
ax = []
for i in range(n_vars):
    if N_H < 3:
        ax.append([fig.add_subplot(gs[i, :3]), fig.add_subplot(gs[i, 3])])
    else:
        ax.append([fig.add_subplot(gs[i, :4]), fig.add_subplot(gs[i, 4:])])
if N_H < 3:
    ax.append([fig.add_subplot(gs[-1, :2]), fig.add_subplot(gs[-1, 2:])])
else:
    ax.append([fig.add_subplot(gs[-1, :2]), fig.add_subplot(gs[-1, 2:4]), fig.add_subplot(gs[-1, 4:])])

col = [[.2,.2,.2], [.8,0,0], [0,.7,0]]

bus_ID = rec_bus_IDs[0]
line_IDs = rec_bus_IDs[0], rec_bus_IDs[0] + 1

idx = t < 60

dm = np.max(measure_exact) - np.min(measure_exact)
ylim = [np.min(measure_exact) - dm / 2, np.max(measure_exact) + dm / 2]

for i in range(N_H):
    for j,var_name in enumerate(var_names_fmt):
        if 'bus' in var_name:
            key = var_name.format(bus_ID)
        elif 'line' in var_name:
            key = var_name.format(line_IDs[0], line_IDs[1])
        value = data_normalized[i][key]
        n,edges = np.histogram(value, bins=25, range=(-4,4), density=True)
        ax[j][0].plot(t[idx], value[idx], color=col[i], lw=1, \
                      label=f'{abbrv[area_measure]} = {measure_exact[i]:.2f} {measure_units}')
        ax[j][1].plot(n, edges[:-1] + np.diff(edges[:2])[0] / 2, color=col[i], lw=1)
        for a in ax[j]:
            a.set_ylim([-4,4])
        ax[j][0].set_ylabel(key)
    ax[-1][i].plot(time / 60, measure[i], 'k', lw=1)
    ax[-1][i].plot(time[[0,-1]] / 60, measure_exact[i] + np.zeros(2), 'k--', lw=1)
    ax[-1][i].set_ylim(ylim)
    ax[-1][i].set_xlabel('Time [min]')

for a in ax:
    for side in 'right','top':
        for aa in a:
            aa.spines[side].set_visible(False)

ax[0][0].legend(loc='best')
ax[-1][0].set_ylim([0.2, 0.25])
for i in range(1, N_H):
    ax[-1][i].set_xlim(ax[-1][0].get_xlim())
    ax[-1][i].set_ylim(ax[-1][0].get_ylim())
#     ax[-1][0].get_shared_x_axes().join(ax[-1][0], ax[-1][i])

# ax[-1][0].set_xlim([0,30])
ax[-1][0].set_ylabel(f'{area_measure.capitalize()} [{measure_units}]')
ax[-2][0].set_xlabel('Time [s]')
ax[-2][1].set_xlabel('Fraction')
fig.tight_layout()
# output_filename = f'IEEE39_area{area_ID}_H_G1={H_G1}_' + \
#     f'rec_buses={rec_bus_list}_load_buses={stoch_load_bus_list}_const_H_{experiment_ID[:6]}.pdf'
# fig.savefig(output_filename)

#### single file

In [ ]:
data_file = 'ieee39_PF_stoch_loads_compensators_5.000_4.330-3.930-4.730_4.470-4.870-4.070_' + \
    '3.570_4.330_4.350_3.770_3.470_3.450_4.200_0.100_0.100_0.100.h5'
# data_file = 'ieee39_PF_stoch_loads_compensators_5.000_4.330-3.830-4.830_4.470-3.970-4.970_' + \
#     '3.570_4.330_4.350_3.770_3.470_3.450_4.200_0.100_0.100_0.100.h5'
t, data, data_norm, data_sliding, _ = load_data_slide([os.path.join(data_dir, data_file)],
                                                    var_names,
                                                    None,
                                                    data_std,
                                                    window_dur,
                                                    window_step,
                                                    add_omega_ref = False,
                                                    verbose = True)
dt = np.diff(t[:2])[0]
time, HH, _ = predict(model, data_sliding, window_step)

In [ ]:
plt.plot(time, HH)
plt.ylim([0.2,0.3])

## Constant measure while varying a compensator's inertia

In [ ]:
change_compensator = True

H_values = [default_H_with_comp]
if change_compensator:
    H_values.append(
        OrderedDict([
            ('G01', 5.00), ('G02', 4.33), ('G03', 4.47), ('G04', 3.57), ('G05', 4.33),
            ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.20),
            ('Comp11', 4.0), ('Comp21', 0.1), ('Comp31', 0.1)
        ])
    )
    H_values.append(
        OrderedDict([
            ('G01', 5.00), ('G02', 4.33), ('G03', 4.47), ('G04', 3.57), ('G05', 4.33),
            ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
            ('Comp11', 8.0), ('Comp21', 0.1), ('Comp31', 0.1)
        ])
    )
else:
    H_values.append(
        OrderedDict([
            ('G01', 5.00), ('G02', 4.58), ('G03', 4.72), ('G04', 3.57), ('G05', 4.33),
            ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.20),
            ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
        ])
    )
    H_values.append(
        OrderedDict([
            ('G01', 5.00), ('G02', 4.83), ('G03', 4.97), ('G04', 3.57), ('G05', 4.33),
            ('G06', 4.35), ('G07', 3.77), ('G08', 3.47), ('G09', 3.45), ('G10', 4.2),
            ('Comp11', 0.1), ('Comp21', 0.1), ('Comp31', 0.1)
        ])
    )

N_H = len(H_values)

measure_exact = []

data_normalized = []
measure = []
area_inertia = []

for H in H_values:
    data_file = data_dir + '/ieee39_PF_stoch_loads_compensators_' + \
        '_'.join(map(lambda h: f'{h:.3f}', H.values())) + '.h5'
    print(f'Reading data from {data_file}.')
    _,_,v,_ = read_area_values(data_file, generators_areas_map_with_comp['default'],
                               P_nom_with_comp, area_measure)
    _,_,h,_ = read_area_values(data_file, generators_areas_map_with_comp['default'],
                               P_nom_with_comp, 'inertia')
    measure_exact.append(v[area_ID - 1])
    area_inertia.append(h[area_ID - 1])

    t, _, data_norm, data_sliding, _ = load_data_slide([data_file],
                                                        var_names,
                                                        None,
                                                        data_std,
                                                        window_dur,
                                                        window_step,
                                                        add_omega_ref = False,
                                                        verbose = True)
    print(f'Duration of the simulation: {t[-1]} sec.')
    data_normalized.append(data_norm)
    dt = np.diff(t[:2])[0]
    time, HH, _ = predict(model, data_sliding, window_step)
    measure.append(HH)
measure_exact = np.array(measure_exact)
area_inertia = np.array(area_inertia)
measure_predicted = np.array(list(map(np.nanmean, measure)))

In [ ]:
area_inertia

In [ ]:
measure_exact

In [ ]:
measure_predicted

In [ ]:
n_vars = len(var_names_fmt)
fig = plt.figure(figsize=(8, n_vars * 2.5))
if N_H < 3:
    gs = fig.add_gridspec(n_vars+1, 4)
elif N_H == 3:
    gs = fig.add_gridspec(n_vars+1, 6)
else:
    raise Exception('Supported values of N_H are 1, 2 or 3')
ax = []
for i in range(n_vars):
    if N_H < 3:
        ax.append([fig.add_subplot(gs[i, :3]), fig.add_subplot(gs[i, 3])])
    else:
        ax.append([fig.add_subplot(gs[i, :4]), fig.add_subplot(gs[i, 4:])])
if N_H < 3:
    ax.append([fig.add_subplot(gs[-1, :2]), fig.add_subplot(gs[-1, 2:])])
else:
    ax.append([fig.add_subplot(gs[-1, :2]), fig.add_subplot(gs[-1, 2:4]), fig.add_subplot(gs[-1, 4:])])

col = [[.2,.2,.2], [.8,0,0], [0,.7,0]]

bus_ID = rec_bus_IDs[0]
line_IDs = rec_bus_IDs[0], rec_bus_IDs[0] + 1

idx = t < 60

dm = np.max(measure_exact) - np.min(measure_exact)
ylim = [np.min(measure_exact) - dm / 2, np.max(measure_exact) + dm / 2]

for i in range(N_H):
    for j,var_name in enumerate(var_names_fmt):
        if 'bus' in var_name:
            key = var_name.format(bus_ID)
        elif 'line' in var_name:
            key = var_name.format(line_IDs[0], line_IDs[1])
        value = data_normalized[i][key]
        n,edges = np.histogram(value, bins=25, range=(-4,4), density=True)
        ax[j][0].plot(t[idx], value[idx], color=col[i], lw=1, \
                     label=f'{abbrv[area_measure]} = {measure_exact[i]:.2f} {measure_units}')
        ax[j][1].plot(n, edges[:-1] + np.diff(edges[:2])[0] / 2, color=col[i], lw=1)
        for a in ax[j]:
            a.set_ylim([-4,4])
        ax[j][0].set_ylabel(key)
    ax[-1][i].plot(time / 60, measure[i], 'k', lw=1)
    ax[-1][i].plot(time[[0,-1]] / 60, measure_exact[i] + np.zeros(2), 'k--', lw=1)
    ax[-1][i].set_ylim(ylim)
    ax[-1][i].set_xlabel('Time [min]')

for a in ax:
    for side in 'right','top':
        for aa in a:
            aa.spines[side].set_visible(False)

ax[0][0].legend(loc='best')
for i in range(1, N_H):
    ax[-1][0].get_shared_x_axes().join(ax[-1][0], ax[-1][i])

# ax[-1][0].set_xlim([0,30])
ax[-1][0].set_ylabel(f'{area_measure.capitalize()} [{measure_units}]')
ax[-2][0].set_xlabel('Time [s]')
ax[-2][1].set_xlabel('Fraction')
fig.tight_layout()
# output_filename = f'IEEE39_area{area_ID}_H_G1={H_G1}_' + \
#     f'rec_buses={rec_bus_list}_load_buses={stoch_load_bus_list}_const_H_{experiment_ID[:6]}.pdf'
# fig.savefig(output_filename)